# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Generating-simple-xml-from-XSD" data-toc-modified-id="Generating-simple-xml-from-XSD-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Generating simple xml from XSD</a></div><div class="lev1 toc-item"><a href="#USAddress" data-toc-modified-id="USAddress-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>USAddress</a></div><div class="lev1 toc-item"><a href="#2.-Setting-attributes" data-toc-modified-id="2.-Setting-attributes-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>2. Setting attributes</a></div><div class="lev1 toc-item"><a href="#3.-Complex-(a.k.a.-multi-document)-schema" data-toc-modified-id="3.-Complex-(a.k.a.-multi-document)-schema-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>3. Complex (a.k.a. multi-document) schema</a></div>

# Generating simple xml from XSD

`address.xsd` is a simple XSD (XML Schema Definition):

In [2]:
# %load address.xsd
<xsd:schema xmlns:xsd="http://www.w3.org/2001/XMLSchema">
  <xsd:complexType name="Address">
    <xsd:sequence>
      <xsd:element name="name"   type="xsd:string"/>
      <xsd:element name="street" type="xsd:string"/>
      <xsd:element name="city"   type="xsd:string"/>
    </xsd:sequence>
  </xsd:complexType>

  <xsd:complexType name="USAddress">
    <xsd:complexContent>
      <xsd:extension base="Address">
        <xsd:sequence>
          <xsd:element name="state" type="USState"/>
          <xsd:element name="zip"   type="xsd:positiveInteger"/>
        </xsd:sequence>
        <xsd:attribute name="country" type="xsd:NMTOKEN" fixed="US"/>
      </xsd:extension>
    </xsd:complexContent>
  </xsd:complexType>

  <xsd:complexType name="UKAddress">
    <xsd:complexContent>
      <xsd:extension base="Address">
        <xsd:sequence>
          <xsd:element name="postcode" type="UKPostcode"/>
        </xsd:sequence>
        <attribute name="exportCode" type="xsd:positiveInteger" fixed="1"/>
      </xsd:extension>
    </xsd:complexContent>
  </xsd:complexType>

  <!-- other Address derivations for more countries -->

  <xsd:simpleType name="USState">
    <xsd:restriction base="xsd:string">
      <xsd:enumeration value="AK"/>
      <xsd:enumeration value="AL"/>
      <xsd:enumeration value="AR"/>
      <xsd:enumeration value="AZ"/>
      <!-- and so on ... -->
    </xsd:restriction>
  </xsd:simpleType>

  <!-- simple type definition for UKPostcode -->
  <!-- *** pyxb mod: provide missing STD *** -->
  <xsd:simpleType name="UKPostcode">
    <xsd:restriction base="xsd:string">
      <xsd:pattern value="[A-Z]{2}\d\s\d[A-Z]{2}"/>
    </xsd:restriction>
  </xsd:simpleType>

</xsd:schema>


SyntaxError: invalid syntax (<ipython-input-2-4f8896348fe7>, line 2)

To generate `address` XML element we should first generate Python binding classes:

In [3]:
%%bash
rm address.py # to prevent conflicts
pyxbgen_ru -u address.xsd -m address

Python for AbsentNamespace0 requires 1 modules


Let's also define function for pretty printing XML schemas:

In [4]:
from lxml import etree

def pprint(x):
    print(etree.tostring(etree.fromstring(x)
                         ,xml_declaration="xml_1.0"
                         ,pretty_print=True
                         ,encoding="utf-8"
                        ).decode("utf-8")    
          
         )

We are now reade to generate several elements from our schema depending on our needs:

### 1. Generic Address

`Address` element without particular specification weather it belongs to US or UK:

In [5]:
import address

addr = address.Address()
addr.name = 'Robert Smith'
addr.street = '8 Oak Avenue'
addr.city = 'Anytown'
addr.state = 'AK'
addr.zip = 12341

pprint(addr.toxml(element_name="Address"))

<?xml version='1.0' encoding='utf-8'?>
<Address>
  <name>Robert Smith</name>
  <street>8 Oak Avenue</street>
  <city>Anytown</city>
</Address>



# USAddress

In [6]:
addr = address.USAddress()
addr.name = 'Robert Smith'
addr.street = '8 Oak Avenue'
addr.city = 'Anytown'
addr.state = 'AK'
addr.zip = 12341

pprint(addr.toxml(element_name="USAddress"))

<?xml version='1.0' encoding='utf-8'?>
<USAddress>
  <name>Robert Smith</name>
  <street>8 Oak Avenue</street>
  <city>Anytown</city>
  <state>AK</state>
  <zip>12341</zip>
</USAddress>



Couple of takeways:  
- `address` schema is of `ComplexType`; thus, not having a `root element` in our schema, we
need to specify `element_name` explicitely in `to_xml` method  
- with this schema definition we can generate several xml documents:  
    - for generic Address  
    - for USAddress   
    - for UKAddress     
    
Assuming we follow the order, in which the elements of USAddress are defined, we can simplify further:

In [7]:
addr = address.USAddress('Robert Smith', '8 Oak Avenue', 'Anytown', 'AK', 12341)

pprint(addr.toxml(element_name='USAddress'))

<?xml version='1.0' encoding='utf-8'?>
<USAddress>
  <name>Robert Smith</name>
  <street>8 Oak Avenue</street>
  <city>Anytown</city>
  <state>AK</state>
  <zip>12341</zip>
</USAddress>



# 2. Setting attributes  

Attributes are set using keywords:

In [8]:
%%bash
rm po4.py
rm address.py
pyxbgen_ru -u po4.xsd -m po4 -u nsaddress.xsd -m address

Python for URN:address requires 2 modules


In [9]:
import importlib
importlib.reload(address)

<module 'address' from '/home/sergey/Py_PyXB/address.py'>

In [10]:
import pyxb
import po4
import pyxb.binding.datatypes as xs

po = po4.purchaseOrder(orderDate=xs.date(1999, 10, 20))
po.shipTo = address.USAddress('Alice Smith', '123 Maple Street', 'Anytown', 'AK', 12341)
po.billTo = address.USAddress('Robert Smith', '8 Oak Avenue', 'Anytown', 'AK', 12341)

pyxb.RequireValidWhenGenerating(False)
pprint(po.toxml("utf-8"))

<?xml version='1.0' encoding='utf-8'?>
<ns1:purchaseOrder xmlns:ns1="URN:purchase-order" orderDate="1999-10-20">
  <ns1:shipTo>
    <name>Alice Smith</name>
    <city>Anytown</city>
    <zip>12341</zip>
    <state>AK</state>
    <street>123 Maple Street</street>
  </ns1:shipTo>
  <ns1:billTo>
    <name>Robert Smith</name>
    <city>Anytown</city>
    <zip>12341</zip>
    <state>AK</state>
    <street>8 Oak Avenue</street>
  </ns1:billTo>
</ns1:purchaseOrder>



# 3. Complex (a.k.a. multi-document) schema

In [11]:
# %load po4.xsd
<xsd:schema xmlns:xsd="http://www.w3.org/2001/XMLSchema"
   targetNamespace="URN:purchase-order"
   xmlns:tns="URN:purchase-order"
   xmlns:address="URN:address"
   elementFormDefault="qualified">
  <xsd:import namespace="URN:address" schemaLocation="nsaddress.xsd"/>
  <xsd:element name="purchaseOrder" type="tns:PurchaseOrderType"/>
  <xsd:element name="comment" type="xsd:string"/>
  <xsd:complexType name="PurchaseOrderType">
    <xsd:sequence>
      <xsd:element name="shipTo" type="address:USAddress"/>
      <xsd:element name="billTo" type="address:USAddress"/>
      <xsd:element ref="tns:comment" minOccurs="0"/>
      <xsd:element name="items"  type="tns:Items"/>
    </xsd:sequence>
    <xsd:attribute name="orderDate" type="xsd:date"/>
  </xsd:complexType>
  <xsd:complexType name="Items">
    <xsd:sequence>
      <xsd:element name="item" minOccurs="0" maxOccurs="unbounded">
        <xsd:complexType>
          <xsd:sequence>
            <xsd:element name="productName" type="xsd:string"/>
            <xsd:element name="quantity">
              <xsd:simpleType>
                <xsd:restriction base="xsd:positiveInteger">
                  <xsd:maxExclusive value="100"/>
                </xsd:restriction>
              </xsd:simpleType>
            </xsd:element>
            <xsd:element name="USPrice"  type="xsd:decimal"/>
            <xsd:element ref="tns:comment"   minOccurs="0"/>
            <xsd:element name="shipDate" type="xsd:date" minOccurs="0"/>
          </xsd:sequence>
          <xsd:attribute name="partNum" type="tns:SKU" use="required"/>
        </xsd:complexType>
      </xsd:element>
    </xsd:sequence>
  </xsd:complexType>
  <!-- Stock Keeping Unit, a code for identifying products -->
  <xsd:simpleType name="SKU">
    <xsd:restriction base="xsd:string">
      <xsd:pattern value="\d{3}-[A-Z]{2}"/>
    </xsd:restriction>
  </xsd:simpleType>

</xsd:schema>


SyntaxError: invalid syntax (<ipython-input-11-4bd644649927>, line 2)

In [12]:
%%bash
rm address.py
rm po4.py
pyxbgen_ru -u po4.xsd -m po4 -u nsaddress.xsd -m address

Python for URN:address requires 2 modules


In [13]:
import pyxb
import po4
import address
import datetime
import pyxb.binding.datatypes as xs
pyxb.utils.domutils.BindingDOMSupport.DeclareNamespace(po4.Namespace, 'FinMon')

po = po4.purchaseOrder()
po.orderDate = xs.date(2016, 1, 13)
po.shipTo = address.USAddress('Alice Smith', '123 Maple Street', 'Anytown', 'AK', 12341)
po.billTo = address.USAddress('Robert Smith', '8 Oak Avenue', 'Anytown', 'AK', 12341)

po.items = pyxb.BIND(pyxb.BIND("Necklace", 4 , 30.0, partNum="798-DF")
                    ,pyxb.BIND("Cockless", 1 , 25.0, partNum="798-FD")
                    )

po.shipTo.country = po.billTo.country = po.shipTo.country

necklace = po.items.item[0]
necklace.shipDate = po.orderDate + datetime.timedelta(days=46)
necklace.comment = 'Want this for the holidays!'

po.items.item[1].shipDate = po.items.item[0].shipDate + datetime.timedelta(days=19)

pprint(po.toxml())

<?xml version='1.0' encoding='utf-8'?>
<FinMon:purchaseOrder xmlns:FinMon="URN:purchase-order" orderDate="2016-01-13">
  <FinMon:shipTo country="US">
    <name>Alice Smith</name>
    <city>Anytown</city>
    <zip>12341</zip>
    <state>AK</state>
    <street>123 Maple Street</street>
  </FinMon:shipTo>
  <FinMon:billTo country="US">
    <name>Robert Smith</name>
    <city>Anytown</city>
    <zip>12341</zip>
    <state>AK</state>
    <street>8 Oak Avenue</street>
  </FinMon:billTo>
  <FinMon:items>
    <FinMon:item partNum="798-DF">
      <FinMon:shipDate>2016-02-28</FinMon:shipDate>
      <FinMon:productName>Necklace</FinMon:productName>
      <FinMon:comment>Want this for the holidays!</FinMon:comment>
      <FinMon:quantity>4</FinMon:quantity>
      <FinMon:USPrice>30.0</FinMon:USPrice>
    </FinMon:item>
    <FinMon:item partNum="798-FD">
      <FinMon:shipDate>2016-03-18</FinMon:shipDate>
      <FinMon:productName>Cockless</FinMon:productName>
      <FinMon:quantity>1</FinMon:quant

In [16]:
po.items.item.append(pyxb.BIND("Neck", 4 , 30.0, partNum="798-DD"))

In [19]:
print(po.toDOM().toprettyxml(encoding="utf-8").decode("utf-8"))

<?xml version="1.0" encoding="utf-8"?>
<FinMon:purchaseOrder orderDate="2016-01-13" xmlns:FinMon="URN:purchase-order">
	<FinMon:shipTo country="US">
		<name>Alice Smith</name>
		<city>Anytown</city>
		<zip>12341</zip>
		<state>AK</state>
		<street>123 Maple Street</street>
	</FinMon:shipTo>
	<FinMon:billTo country="US">
		<name>Robert Smith</name>
		<city>Anytown</city>
		<zip>12341</zip>
		<state>AK</state>
		<street>8 Oak Avenue</street>
	</FinMon:billTo>
	<FinMon:items>
		<FinMon:item partNum="798-DF">
			<FinMon:shipDate>2016-02-28</FinMon:shipDate>
			<FinMon:productName>Necklace</FinMon:productName>
			<FinMon:comment>Want this for the holidays!</FinMon:comment>
			<FinMon:quantity>4</FinMon:quantity>
			<FinMon:USPrice>30.0</FinMon:USPrice>
		</FinMon:item>
		<FinMon:item partNum="798-FD">
			<FinMon:shipDate>2016-03-18</FinMon:shipDate>
			<FinMon:productName>Cockless</FinMon:productName>
			<FinMon:quantity>1</FinMon:quantity>
			<FinMon:USPrice>25.0</FinMon:USPrice>
		</FinMo